<a href="https://colab.research.google.com/github/DaverArenas/StockExchange_Predictions/blob/main/Random_Forest_With_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://www.kaggle.com/code/musabbirarrafi/random-forest-with-optuna

Based on this submission https://www.kaggle.com/code/listeningtounity/random-forest-with-optuna¶

In [1]:
import numpy as np
import pandas as pd